<a href="https://colab.research.google.com/github/tarakantaacharya/Stock_Movement_Analysis/blob/main/DataScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Scraping

###Instructions to run code :
Due to security purposes in Github the API details are not given. So you have to create API key and use it into reddit API section then you can run the code without any interruption

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

The command !pip install praw is used to install the praw Python library using the package manager pip.

PRAW allows developers to access Reddit data programmatically. With PRAW, you can:

Retrieve posts and comments from subreddits.
Submit posts and comments.
Extract information for data analysis, sentiment analysis, or natural language processing (NLP) projects.
Automate interactions with Reddit, such as monitoring specific subreddits.

In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.6 MB/s eta 0:00:00


Importing the libraries

In [ ]:
# Importing PRAW (Python Reddit API Wrapper) to interact with Reddit's API
import praw
# Importing pandas to work with data in DataFrame format (useful for data manipulation and analysis)
import pandas as pd
# Importing requests to handle HTTP requests and catch potential 403 errors (forbidden access)
import requests  # Useful when accessing web resources or handling HTTP errors

For the security purpose in GitHub, The client Id, client_secret and user_agent are not provided. You can use your client_id,client_secret and user_agent from Reddit API  

In [ ]:
# Initialize Reddit API
reddit = praw.Reddit(
    client_id='#################',  # Your client ID
    client_secret='########################',  # Your client secret
    user_agent='################################', # Your user agent
    check_for_async=False #Recommended to use async PRAW for scrapping
)

# List of subreddits focused on stock market discussions
subreddits = [
    'WallStreetBets', 'stocks', 'StockMarket', 'investing', 'pennystocks', 'finance',
    'options', 'ValueInvesting', 'DividendInvesting', 'shortsqueeze', 'Trading',
    'Investing101', 'FI', 'financialindependence', 'cryptocurrency', 'Economics',
    'StockGains', 'StocksForBeginners', 'BitcoinMarkets', 'Fundamentals', 'stockpicks',
    'TradeShares', 'MomentumStocks', 'TechStocks', 'BigStocks', 'FinTech', 'ETFs',
    'Actuaries', 'InvestingInBonds', 'TradingAlgorithms'
]

In [ ]:
# Initialize empty list to store post data
posts_data = []

# Function to handle 403 errors by skipping the subreddit
def handle_403_error(subreddit_name):
    print(f"403 error encountered for subreddit: {subreddit_name}. Skipping this subreddit.")
    return True  # Return True to indicate the subreddit should be skipped

# Loop through each subreddit and collect posts
for subreddit_name in subreddits:
    try:
        subreddit = reddit.subreddit(subreddit_name)

        # Check if we can access the subreddit (avoid 403 error)
        subreddit._fetch()  # Try fetching subreddit data to catch any 403 errors

        # Fetch top posts (you can change this to other sorting methods)
        for post in subreddit.top(limit=5000):  # Fetch top 10 posts (can adjust limit as needed)
            post_data = {
                'subreddit': subreddit_name,
                'title': post.title,
                'content': post.selftext,
                'score': post.score,
                'num_comments': post.num_comments,
                'url': post.url,
                'created_utc': datetime.utcfromtimestamp(post.created_utc),
                'upvote_ratio': post.upvote_ratio,
                'author': post.author.name if post.author else 'N/A'
            }

            posts_data.append(post_data)

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 403:
            if handle_403_error(subreddit_name):  # Skip this subreddit
                continue  # Skip the current subreddit and go to the next one

    except Exception as e:
        print(f"An error occurred while processing subreddit '{subreddit_name}': {e}")
        continue  # Skip to the next subreddit if an error occurs

    # Add sleep to avoid rate limiting issues
    time.sleep(2)

# Create a DataFrame to store the posts
df_posts = pd.DataFrame(posts_data)

# Save to CSV (you can change this to JSON, or any format you prefer)
df_posts.to_csv('reddit_stock_data_posts.csv', index=False)

print("Data collection complete. Saved to 'reddit_stock_data_posts.csv'.")

An error occurred while processing subreddit 'FI': received 404 HTTP response
An error occurred while processing subreddit 'TradeShares': Redirect to /subreddits/search
An error occurred while processing subreddit 'MomentumStocks': Redirect to /subreddits/search
An error occurred while processing subreddit 'BigStocks': Redirect to /subreddits/search
An error occurred while processing subreddit 'InvestingInBonds': Redirect to /subreddits/search
An error occurred while processing subreddit 'TradingAlgorithms': received 404 HTTP response
Data collection complete. Saved to 'reddit_stock_data_posts.csv'.


In [ ]:
fd = pd.read_csv('reddit_stock_data_posts.csv')   #The scrapped dataframe is converted into csv file and stored for next process
fd.head()                                         #It will give top 5 rows of dataframe

,subreddit,title,content,score,num_comments,url,created_utc,upvote_ratio,author
0,WallStreetBets,Times Square right now,NaN,489193,14013,https://v.redd.it/x64z70f7eie61,2021-01-30 18:00:38,0.99,SomeGuyInDeutschland
1,WallStreetBets,UPVOTE so everyone sees we got SUPPORT,NaN,338563,12843,https://i.redd.it/sgoqy8nyt2e61.png,2021-01-28 13:40:34,0.98,vrweensy
2,WallStreetBets,GME YOLO update — Jan 28 2021,NaN,300871,23007,https://i.redd.it/opzucppb15e61.png,2021-01-28 21:06:23,0.98,DeepFuckingValue
3,WallStreetBets,GME YOLO month-end update — Jan 2021,NaN,264904,19896,https://i.redd.it/r557em3t5ce61.png,2021-01-29 21:04:45,0.98,DeepFuckingValue
4,WallStreetBets,It’s treason then,NaN,247637,4596,https://i.redd.it/d3t66lv1yce61.jpg,2021-01-29 23:40:59,0.98,keenfeed


In [ ]:
fd.shape    #Determine the shape of Dataframe "fd"(scrapped dataframe)

(16947, 9)

The Scrapped data has 16947 rows and 9 columns !!!

In [ ]:
fd.columns   #Column names of scrapped data

Index(['subreddit', 'title', 'content', 'score', 'num_comments', 'url',
       'created_utc', 'upvote_ratio', 'author'],
      dtype='object')

Here we scrapped data from reddit and stored in respective dataframe and csv files. Next we will go to Data preprocessing step where we add some columns using this scrapped data columns and also clean the data for better prediction